In [49]:
import pandas as pd
import sqlite3 as sql
import numpy as np

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [3]:
places_df = pd.read_csv('../data/foursquare_bikes.csv')
stations_df = pd.read_csv('../data/berlin_bikes.csv')

print(places_df.columns)
print(stations_df.columns)
# We will have to join on places_df.origin and a new column in stations_df of lat+','+long
stations_df['origin'] = stations_df['lat'].astype(str) + ',' + stations_df['long'].astype(str)
stations_df = stations_df.drop(['lat','long','Unnamed: 0'], axis=1)

places_df = places_df.drop('Unnamed: 0',axis=1)
places_df['origin'] = places_df['origin'].astype(str)
stations_df['origin'] = stations_df['origin'].astype(str)

Index(['Unnamed: 0', 'name', 'distance_meters', 'category', 'popularity',
       'rating', 'lat', 'long', 'origin'],
      dtype='object')
Index(['Unnamed: 0', 'lat', 'long', 'bikes_available', 'bikes_in_use',
       'total_bikes'],
      dtype='object')


In [4]:
places_df.head(1)

,name,distance_meters,category,popularity,rating,lat,long,origin
0,Curry Wolf,29.0,Fast Food Restaurant,0.903641,7.8,52.503902,13.335662,"52.504157,13.335328"


In [5]:
stations_df.head(1)

,bikes_available,bikes_in_use,total_bikes,origin
0,4,0,4,"52.504157,13.335328"


In [6]:
bike_poi_df = pd.merge(places_df,stations_df,on='origin')

In [7]:
bike_poi_df.groupby(by='origin').head(10)

,name,distance_meters,category,popularity,rating,lat,long,origin,bikes_available,bikes_in_use,total_bikes
0,Curry Wolf,29.0,Fast Food Restaurant,0.903641,7.8,52.503902,13.335662,"52.504157,13.335328",4,0,4
1,Pull&Bear Ber-Tauentzienstrasse,29.0,Clothing Store,0.988963,7.8,52.504021,13.335963,"52.504157,13.335328",4,0,4
2,Levi Strauss Germany GmbH,19.0,Clothing Store,0.936147,6.7,52.504051,13.334821,"52.504157,13.335328",4,0,4
3,Commerzbank,13.0,Bank,0.995382,6.0,52.504245,13.334703,"52.504157,13.335328",4,0,4
4,Curry Am Ku'damm,10.0,Snack Place,0.781733,NaN,52.504211,13.334964,"52.504157,13.335328",4,0,4
...,...,...,...,...,...,...,...,...,...,...,...
46163,Nice Ice,31.0,Ice Cream Parlor,0.382586,NaN,52.510668,13.381195,"52.51104,13.38126",1,3,4
46164,zero Textilhandel,16.0,Fabric / Textiles Store,NaN,NaN,52.510934,13.381434,"52.51104,13.38126",1,3,4
46165,Edeka,55.0,Grocery Store / Supermarket,0.954706,7.2,52.510483,13.380640,"52.51104,13.38126",1,3,4
46166,Private Textiles G. M. B. H.,19.0,Clothing Store,NaN,NaN,52.510869,13.381180,"52.51104,13.38126",1,3,4


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [161]:
# This is by far the one I had the most fun with and it provides excellent easily digestible context for the data.
# plotly.express has a really cool feature where you can scatter plots onto a detailed interactive map.

import plotly.express as px
tmp_stations_df = pd.read_csv('../data/berlin_bikes.csv')
# set x and y
x = tmp_stations_df['lat']
y = tmp_stations_df['long']

fig = px.scatter_mapbox(tmp_stations_df,lat=x,lon=y,zoom=10,height=1000,width=1600,color='total_bikes',size='bikes_in_use', opacity=1)
#                       df we're using, x,      y   ------self explanitory-------- set gradient and size for spectrum of values in set column.
fig.update_layout(mapbox_style='open-street-map')
fig.show()

# This first plot is pretty basic. we're just plotting the location of every station, darker colored stations have fewer bikes and brighter stations have more bikes. Smaller stations have fewer bikes being used and larger have more.

From this first plot there's quite a few patterns we can pick out.
* In the city center there's low total bike stations distributed in a dense grid pattern. In a lot of the city there's one on every corner.
* The biggest bike stations are most often by parks/nature or plazas/old city shopping centres. Parks and Nature paths aren't picked up in the foursquares search so there's some neglect of relevant information there.
    * The north east station in the park by the campus is massive and very busy. The other bike stations, especially by the dorms are also busy.
    * In the heart of Berlin (GeorgenStrase) There's a big, very busy, bike station that likely has a route along the river. This is probably a big tourist spot.
    * However in the west, out of the city, there's a large park called Grunewald that isn't that popular for biking. I wonder why that is. Could this be used more by locals and not tourists?
* While the outskirts of Berlin don't have a lot of stations there's some locality's that have quite a few. This presents a pattern in their growth out of mostly a residential community of Berlin into more of a little city of its own. (Tegel, Rummelburg and furthest away is Hellendorf).

In [160]:
tmp_places = places_df
# split origin (station lat long) into two columns to it's easier to set x and y.
tmp_places[['new_lat', 'new_long']] = places_df['origin'].str.split(',', expand=True)

# filter for stations that were unable to find places of interest in a 200m radius
no_poi = tmp_places[pd.isna(tmp_places['lat'])]

# set x and y
x = no_poi['new_lat']
y = no_poi['new_long']
x = x.astype(float)
y = y.astype(float)
# origin is a string, they need to be floats.

# basic plot
fig = px.scatter_mapbox(no_poi,lat=x,lon=y,zoom=10,height=1000,width=1600, opacity=1)
fig.update_layout(mapbox_style='open-street-map')
fig.show()

In [155]:

places_grouped = places_df.groupby(['lat', 'long']).size().reset_index(name='counts')
x = places_grouped['lat']
y = places_grouped['long']

fig = px.scatter_mapbox(places_grouped,lat=x,lon=y,zoom=10,height=1000,width=1600,color='counts',size='counts', opacity=0.6)
fig.update_layout(mapbox_style='open-street-map')
fig.show()

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [9]:
bike_poi_df.to_sql('../data/bike_poi_df.sql')

TypeError: to_sql() missing 1 required positional argument: 'con'

Look at the data before and after the join to validate your data.